In [20]:
import numpy as np
import os
import tensorflow as tf

In [2]:
data_dir="Dataset_l/Plant_Disease/"
train_dir=os.path.join(data_dir,"train")
val_dir=os.path.join(data_dir,"validation")
test_dir=os.path.join(data_dir,"test")

In [3]:
labels=os.listdir(train_dir)
print(labels)


['grape', 'tomato', 'corn (maize)', 'peach', 'squash', 'Rice', 'apple', 'Cassava', 'cherry (including sour)', 'potato', 'strawberry', 'orange', 'pepper, bell']


In [4]:
for i in labels:
    disease=os.path.join(train_dir,i)
    disease=os.listdir(disease)
    print(disease)

['leaf blight (isariopsis leaf spot)', 'healthy', 'esca (black measles)', 'black rot']
['late blight', 'early blight', 'spider mites two-spotted spider mite', 'healthy', 'tomato mosaic virus', 'septoria leaf spot', 'tomato yellow leaf curl virus', 'target spot', 'leaf mold', 'bacterial spot']
['healthy', 'cercospora leaf spot gray leaf spot', 'common rust', 'northern leaf blight']
['healthy', 'bacterial spot']
['powdery mildew']
['BrownSpot', 'LeafBlast', 'Hispa', 'Healthy']
['healthy', 'apple scab', 'cedar apple rust', 'black rot']
['Bacterial Blight (CBB)', 'Green Mottle (CGM)', 'Brown Streak Disease (CBSD)', 'Healthy', 'Mosaic Disease (CMD)']
['healthy', 'powdery mildew']
['late blight', 'early blight', 'healthy']
['leaf scorch', 'healthy']
['haunglongbing (citrus greening)']
['healthy', 'bacterial spot']


In [24]:
#preprocessing and normalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen=[]
test_datagen=[]
validation_datagen=[]
for i in labels:
    train_datagen.append(ImageDataGenerator(
        rescale=1./255,
        rotation_range=0.2,
            horizontal_flip=True
    ))
    test_datagen.append(ImageDataGenerator(rescale=1./255))
    validation_datagen.append(ImageDataGenerator(rescale=1./255))

train_generator=[]
val_generator=[]
test_generator=[]
k=0
for i in train_datagen:
    new_train_dir=os.path.join(train_dir,labels[k])
    train_generator.append(i.flow_from_directory(
        new_train_dir,
        target_size=(256,256),
        color_mode="rgb",
        batch_size=32,
        class_mode='categorical'
    ))
    k+=1
k=0
for i in validation_datagen:
    new_val_dir=os.path.join(val_dir,labels[k])
    val_generator.append(i.flow_from_directory(
        new_val_dir,
        target_size=(256,256),
        color_mode="rgb",
        batch_size=32,
        class_mode='categorical'
    ))
    k+=1
k=0
for i in test_datagen:
    new_test_dir=os.path.join(test_dir,labels[k])
    test_generator=i.flow_from_directory(
        new_test_dir,
        target_size=(256,256),
        color_mode="rgb",
        batch_size=32,
        class_mode='categorical'
    )
    k+=1


Found 2927 images belonging to 4 classes.
Found 13083 images belonging to 10 classes.
Found 2776 images belonging to 4 classes.
Found 1915 images belonging to 2 classes.
Found 1322 images belonging to 1 classes.
Found 2419 images belonging to 4 classes.
Found 2286 images belonging to 4 classes.
Found 1573 images belonging to 5 classes.
Found 1374 images belonging to 2 classes.
Found 1550 images belonging to 3 classes.
Found 1129 images belonging to 2 classes.
Found 3966 images belonging to 1 classes.
Found 1784 images belonging to 2 classes.
Found 730 images belonging to 4 classes.
Found 3265 images belonging to 10 classes.
Found 692 images belonging to 4 classes.
Found 477 images belonging to 2 classes.
Found 330 images belonging to 1 classes.
Found 603 images belonging to 4 classes.
Found 569 images belonging to 4 classes.
Found 346 images belonging to 5 classes.
Found 342 images belonging to 2 classes.
Found 387 images belonging to 3 classes.
Found 281 images belonging to 2 classes.

In [25]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers.legacy import Adam

In [31]:
model=Sequentiamodel = Sequential()
model.add(layers.Conv2D(32,(3,3),activation="relu",padding="same",input_shape=(256,256,3)))
model.add(layers.Conv2D(32,(3,3),activation="relu",padding="same"))
model.add(layers.MaxPooling2D(3,3))

model.add(layers.Conv2D(64,(3,3),activation="relu",padding="same"))
model.add(layers.Conv2D(64,(3,3),activation="relu",padding="same"))
model.add(layers.MaxPooling2D(3,3))

model.add(layers.Conv2D(128,(3,3),activation="relu",padding="same"))
model.add(layers.Conv2D(128,(3,3),activation="relu",padding="same"))
model.add(layers.MaxPooling2D(3,3))

model.add(layers.Flatten())

model.add(layers.Dense(256,activation="relu"))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(3,activation="softmax"))
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 256, 256, 32)      896       
                                                                 
 conv2d_13 (Conv2D)          (None, 256, 256, 32)      9248      
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 85, 85, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 85, 85, 64)        18496     
                                                                 
 conv2d_15 (Conv2D)          (None, 85, 85, 64)        36928     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 28, 28, 64)        0         
 g2D)                                                 

In [41]:
#tb_callback2=tf.keras.callbacks.TensorBoard(log_dir="logs/",histogram_freq=1)
es = EarlyStopping(
    min_delta=0.01,
    patience=5,
    restore_best_weights=True,
)


for i in range(len(train_generator)):
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])


Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node 'gradient_tape/log_cosh/sub/BroadcastGradientArgs' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/fansan/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/fansan/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/fansan/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 736, in start
      self.io_loop.start()
    File "/home/fansan/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
 